### Importando librerías necesarias

In [ ]:
#!pip install missingno

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import missingno as msno

import scipy.stats as stats

### Estructura de datos - Dataset Seoul Bike

*  Date - Día donde se rentaron bicicletas 
*  Rented Bike count - Cantidad de bicicletas arrendadas en cada hora
*  Hour - Hora del día (en formato 0 a 24)
*  Temperature - Temperatura en Celsius
*  Humidity - Porcentaje de humedad
*  Windspeed - Velocidad del viento en m/s
*  Visibility - Visibilidad atmosférica dentro de un rango de 10m
*  Dew point temperature - Temperatura del punto de rocío en Celsius
*  Solar radiation - Radiación Solar MJ/m2
*  Rainfall - mm de lluvia
*  Snowfall - cm de nieve caída 
*  Seasons - Temporada del año en inglés (Verano, Invierno, Otoño, Primavera)
*  Holiday - Flag representando de si es un Festivo o no
*  Functional Day - Flag mostrando si es un día funcional o no

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/patoram123/Proyecto_Visualizacion/main/datasets/public_bikes_2018/seul/SeoulBikeData.csv'
                ,encoding='unicode escape')

df['Date'] = pd.to_datetime(df['Date'],format="%d/%m/%Y")

df.head()

### Revisando cantidad de valores nulos, unicos, duplicados y tipos de datos

In [ ]:
def datainfo(data):
    print(f"Cantidad de columnas duplicadas: {len(data[data.duplicated()])}")
    temp_ps = pd.DataFrame(index=data.columns)
    temp_ps['DataType'] = data.dtypes
    temp_ps["Non-null_Values"] = data.count()
    temp_ps['Unique_Values'] = data.nunique()
    temp_ps['NaN_Values'] = data.isnull().sum()
    return temp_ps

In [ ]:
datainfo(df)

* De aquí podemos notar como la mayoría de las variables son númericas y las variables categóricas están mapeadas como de tipo Object. No tenemos nulos y las variables categoricas cumplen lo esperado en la definición de datos.

#### Gráfico de missing values

In [ ]:
msno.matrix(df,labels=[df.columns],figsize=(30,16),fontsize=12)

### Correlación de Variables númericas

In [ ]:
#matriz de correlacion 
corrMatrix = df.corr()
plt.rcParams['axes.facecolor'] = 'white'

mask = np.triu(np.ones_like(corrMatrix, dtype=bool))

cmap = sns.diverging_palette(250, 15, s=75, l=40,
                           n=9, center="light", as_cmap=True)

fig, ax = plt.subplots(figsize = (16, 8))

sns.heatmap(corrMatrix, mask=mask, center=1, annot=True,
            fmt='.2f', square=True, cmap=cmap)

sns.set(font_scale=0.8)

plt.show()

In [ ]:
print("\033[1m Correlación entre Temperature(°C) y Dew point temperature(°C)")

sns.regplot(x = "Temperature(°C)", 
            y = "Dew point temperature(°C)"  , 
            data = df,
            line_kws={"color": "red"}) 
plt.show()

* Considerando la alta correlación que existe entre las variables `Dew point temperature(°C)` y `Temperature(°C)` se decide eliminar la primera

In [ ]:
df.drop(['Dew point temperature(°C)'], axis = 1, inplace=True)
df.head(1)

### Creación y eliminación de columnas (cont.)

* La columna `Functioning Day`, que toma valores Yes o No, representa si ese día el servicio de arriendo de bicicletas estuvo operativo o no. Para los días no funcionales, la variable `Rented Bike Count` tiene solo valores 0 (cero), por lo que se decidió eliminar esas filas y por lo mismo, la columna `Functioning Day` que deja de tener valor.


* Para la variable fecha se aprovechará de obtener valor al separar en nuevas variables: Día y Mes. Elimando la columna `Date` del Dataset

In [ ]:
print("******* Functioning Day *******")
print(df['Functioning Day'].value_counts())

print("\n\n******* Rented Bikes when Functioning Day is No *******")
print(df[df['Functioning Day']=="No"]['Rented Bike Count'].value_counts())

In [ ]:
# Quitando filas donde Functioning Day is No
df = df[df['Functioning Day']=="Yes"].reset_index(drop=True)
print(df['Functioning Day'].value_counts())

# Quitando la columna fecha ya que no aportará al modelo
df.drop(columns=['Functioning Day'],inplace=True)

In [ ]:
# Creando variables de día y mes
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month

# Quitando la columna fecha ya que no aportará al modelo
df.drop(columns=['Date'],inplace=True)

df.head(1)

### Box plot de Variables categóricas vs Variable dependiente

* En los box plots se puede apreciar la diferencia de las medianas de la variable objetivo respecto a la estación y al tipo de día. En invierno la mediana del arriendo de bicicletas es casi cinco veces menor que en verano, y en días festivos la mediana es casi cuatro veces inferior que en un día normal.

In [ ]:
#Box plot de cada variable independiente v/s la variable dependiente
columns=['Seasons','Holiday']

fig=plt.figure(figsize=(20,20))
for i,cols in enumerate(columns): 
    ax=fig.add_subplot(3,3,i+1)
    sns.boxplot(x=cols,y='Rented Bike Count', data=df,ax=ax)

### Distribución de los datos

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(12, 12))
for i, col in enumerate(df.select_dtypes(include=['float64','int64']).columns):
    plt.rcParams['axes.facecolor'] = 'black'
    ax = plt.subplot(6,2, i+1)
    sns.histplot(data=df, x=col, ax=ax,color='red',kde=True)
plt.suptitle('Data distribution of continuous variables')
plt.tight_layout()

In [ ]:
# Gráfico que permite visualizar la distribución y su cercanía con 
# la normal a través del QQ plot
def plotvariable(df,variable):
    plt.figure(figsize=(10,5))
    plt.rcParams['axes.facecolor'] = 'white'
    plt.subplot(1,2,1)  
    df[variable].hist(bins=30)
    
    ##QQ plot
    plt.subplot(1,2,2)
    stats.probplot(df[variable], dist='norm',plot=plt)
    plt.show()

In [ ]:
plotvariable(df,'Rented Bike Count')

In [ ]:
plotvariable(df,'Wind speed (m/s)')

### Normalizando variables

* Al observar las distribuciones, podemos notar como algunas variables no tienen una distribución cercana a la normal. Existen distintos metodos que nos pueden ayudar a normalizar los datos, entre ellos aplicar transformaciones de potencia. Para el caso de la Variable `Rented Bike Count` y `Wind speed (m/s)`, aplicar una función cuadrática (sqrt) nos deja las distribuciones muy cercanas a la Normal.

In [ ]:
df_norm = df.copy()

In [ ]:
df_norm['Rented Bike Count'] = np.sqrt(df_norm['Rented Bike Count'])
plotvariable(df_norm,'Rented Bike Count')

In [ ]:
df_norm['Wind speed (m/s)']=np.sqrt(df_norm['Wind speed (m/s)'])

plotvariable(df_norm,'Wind speed (m/s)')

### One Hot Encoding para Variables Categoricas

* Para obtener valores numericos de las variables categoricas, se decidió transformarlas usando "Dummyficación" o One Hot Encoding, lo cual implica crear tantas columnas como categorías existan y asignar un 1 donde la variable tome el valor que corresponda.

In [ ]:
df_norm = pd.get_dummies(df_norm, columns=['Seasons', 'Holiday'], drop_first=True)
df_norm.head()

### Exportando Dataset final

In [ ]:
df_norm.to_csv("../data/processed/Clean_SeoulBikeData.csv")